# Documentation for Standard Keras Neural Network
- ROC AUC for different masses
- learning curve
- validation curves

# Standard imports

In [0]:
! wget https://jglatzer.web.cern.ch/jglatzer/hpml/pandas_ge6jge4b.h5

--2019-02-27 13:41:46--  https://jglatzer.web.cern.ch/jglatzer/hpml/pandas_ge6jge4b.h5
Resolving jglatzer.web.cern.ch (jglatzer.web.cern.ch)... 188.184.64.89, 2001:1458:201:a4::100:53
Connecting to jglatzer.web.cern.ch (jglatzer.web.cern.ch)|188.184.64.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83895032 (80M) [text/plain]
Saving to: ‘pandas_ge6jge4b.h5.1’

pandas_ge6jge4b.h5. 100%[===================>]  80.01M  19.0MB/s    in 7.7s    

2019-02-27 13:41:55 (10.4 MB/s) - ‘pandas_ge6jge4b.h5.1’ saved [83895032/83895032]



In [0]:

! git clone https://github.com/JulianGlatzer/HplusML
import sys
sys.path.append("HplusML")
#! export PYTHONPATH=$PYTHONPATH:`pwd`/HplusML
#! echo $PYTHONPATH

Cloning into 'HplusML'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 46 (delta 19), reused 38 (delta 11), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import HpAlgorithms
import HpMLUtils
import HpMLPlots
import HpHyperParameterOptimisation
import HpTrainingFrame

In [0]:
df_mc=pd.read_hdf('pandas_ge6jge4b.h5', 'INC_ge6jge4b')
df_mc=df_mc[df_mc.process.apply(lambda x: True if x in ["ttlight","ttb","ttc","Hp800"] else False)]
#print df_mc.process.shape
df_mc['process'].unique()

array(['ttlight', 'ttc', 'ttb', 'Hp800'], dtype=object)

In [0]:
df_mc.columns

Index([u'index', u'Muu_MindR_70', u'nJets', u'Mbb_MindR_70', u'HT_jets',
       u'H1_all', u'Mjjj_MaxPt', u'Mbb_MaxPt_70', u'dRbb_avg_70',
       u'eventNumber', u'pT_jet5', u'dRlepbb_MindR_70', u'nBTags_70',
       u'Mbb_MaxM_70', u'jet_pt', u'Centrality_all', u'weight', u'process',
       u'group', u'region', u'pT_jet1', u'hpmass'],
      dtype='object')

# Prepare data for H+ classification

split into test and training data

In [0]:
htf=HpTrainingFrame.HpTrainingFrame(df_mc) 

In [0]:
from HpMLUtils import WeightedStandardScaler, summarizeFitData,MultiSBWeightsScaler
X_train, X_test, X_eval, y_train, y_test,y_eval, w_train, w_test, w_eval=htf.prepare(hpmass=800)
summarizeFitData(X_train, y_train, w=w_train)

sb=MultiSBWeightsScaler(backgroundclass=0)
sb.fit(X_train,y_train, sample_weight=w_train)
X_train=sb.transform(X_train,y_train, sample_weight=w_train)
X_test=sb.transform(X_test,y_test, sample_weight=w_test)
#w_train.groupby(y_train).sum()
summarizeFitData(X_train, y_train, w=w_train)

wss=WeightedStandardScaler()
wss.fit(X_train,sample_weight=w_train)
X_train=wss.transform(X_train)
X_test=wss.transform(X_test)
#X_eval=wss.transform(X_eval)
summarizeFitData(X_train, y_train, w=w_train)
summarizeFitData(X_test, y_test, w=w_test)

X.shape= (23068, 14) y.shape= (23068,) w.shape= (23068,)
columns= Index([u'nJets', u'nBTags_70', u'pT_jet1', u'Mbb_MindR_70', u'pT_jet5',
       u'H1_all', u'dRbb_avg_70', u'dRlepbb_MindR_70', u'Muu_MindR_70',
       u'HT_jets', u'Mbb_MaxPt_70', u'Mbb_MaxM_70', u'Mjjj_MaxPt',
       u'Centrality_all'],
      dtype='object')
categories= [0, 1]

sum of weights per category
       all 1464.08896891
         0 1457.10675797
         1 6.98221093499


average
  variable/class        all          0          1
           nJets       6.99       6.99       7.43
       nBTags_70       4.06       4.06        4.1
         pT_jet1   1.77e+05   1.77e+05   2.96e+05
    Mbb_MindR_70   7.76e+04   7.73e+04   1.33e+05
         pT_jet5   5.06e+04   5.05e+04   6.68e+04
          H1_all      0.226      0.227      0.182
     dRbb_avg_70       2.28       2.28       2.42
dRlepbb_MindR_70       2.31       2.31       2.17
    Muu_MindR_70   1.05e+05   1.05e+05   1.06e+05
         HT_jets   5.69e+05   5.68e+05   